# 模型的训练与预测
  首先训练模型并保存，然后在预测的时候读取模型并将结果保存目标格式
    * 模型训练首先要将图片保存为tensowflow所需格式，所以第一次训练的时候很慢，但之后就好了
    * 模型预测要注意图片是否损坏及格式问题


In [1]:
%load_ext autoreload
%autoreload 2

In [1]:
%run retrain.py \
--bottleneck_dir=testbottlenecks \
--how_many_training_steps 1000 \
--learning_rate 0.001 \
--threshold 0.0001 \
--model_dir=model_dir \
--output_graph=retrained_graph.pb \
--output_labels=retrained_labels.txt \
--summaries_dir=retrain_logs \
--image_dir=testimages

INFO:tensorflow:Summary name cross entropy is illegal; using cross_entropy instead.
Instructions for updating:
Use `tf.global_variables_initializer` instead.
2018-04-21 16:15:41.198143: Step 0: Train f1_score = 0.05330
2018-04-21 16:15:41.199143: Step 0: Cross entropy = 0.710947
2018-04-21 16:15:41.482332: Step 0: Validation f1_score = 0.01711
2018-04-21 16:15:44.269613: Step 10: Train f1_score = 0.05573
2018-04-21 16:15:44.269613: Step 10: Cross entropy = 0.710840
2018-04-21 16:15:44.481755: Step 10: Validation f1_score = 0.01693
2018-04-21 16:15:47.440738: Step 20: Train f1_score = 0.06471
2018-04-21 16:15:47.440738: Step 20: Cross entropy = 0.710623
2018-04-21 16:15:47.638873: Step 20: Validation f1_score = 0.04131
2018-04-21 16:15:50.225303: Step 30: Train f1_score = 0.06692
2018-04-21 16:15:50.225303: Step 30: Cross entropy = 0.710422
2018-04-21 16:15:50.462462: Step 30: Validation f1_score = 0.04908
2018-04-21 16:15:53.025970: Step 40: Train f1_score = 0.06906
2018-04-21 16:15:53

KeyboardInterrupt: 

学习速率是0.00005的时候 训练的准确率在50%左右，变化很小,但预测的结果几乎都一样，存在过拟合觉得。

In [1]:
from PredictImage import PredictImage
import os
import pandas as pd
from tqdm  import tqdm
from sklearn.metrics import f1_score
import json
import numpy as np
from sklearn.metrics import classification_report
from sklearn.metrics import precision_recall_fscore_support
from PIL import Image
from os import PathLike
%load_ext autoreload
%autoreload 2

In [7]:
sub_dirs = [x[0] for x in os.walk('./images')]
  # The root directory comes first, so skip it.
is_root_dir = True
for sub_dir in sub_dirs:
    if is_root_dir:
        is_root_dir = False
        continue
    extensions = ['jpg', 'jpeg', 'JPG', 'JPEG']
    file_list = []
    dir_name = os.path.basename(sub_dir)
    print(dir_name)

multi-label


In [2]:
def isValidImage(file): 
    bValid = True
    fileObj = open(file, 'rb')
    buf = fileObj.read()
    if buf[6:10] in (b'JFIF', b'Exif'):     #jpg图片
        if not buf.rstrip(b'\0\r\n').endswith(b'\xff\xd9'):
            bValid = False
    else:        
        try:  
            Image.open(fileObj).verify() 
        except:  
            bValid = False         
    return bValid

In [3]:
def getPredictResult(threshold):
    predictImage = PredictImage()
    pathDir = './test/'
    testImages =  os.listdir(pathDir)
    imageIds = []
    labels = []
    with tqdm(total=len(testImages)) as bar:
        for testImage in testImages:
            imageId = int(testImage[0:-4])
            imageIds.append(imageId)
            imagepath = pathDir + testImage
            if isValidImage(imagepath):
        #print(imagepath)
                l = predictImage.predictResult(imagepath, t = threshold)
                label = ''.join([c + ' ' for c in l])
                a = label.rstrip()
                labels.append(a)
            else:
                labels.append(' ') 
            bar.update(1)
    dataframe = pd.DataFrame({'image_id':imageIds,'label_id':labels})
    dataframe.to_csv('predictResult.csv',sep = ',', index = False)

In [4]:
def strToArray(data):
    out = []
    for i in data:
     if i != '':
        out.append(int(i))
    return out

In [5]:
def getF1Score():
    dfPredict = pd.read_csv("./predictResult.csv")
    dfValidation = pd.read_csv("./validation.csv")
    dfPredict = dfPredict.sort_values(by = 'image_id', ascending=True)
    dfValidation = dfValidation.sort_values(by = 'image_id',ascending = True)
    dfPredict.head()
    dfValidation.head()
    m,n = dfPredict.shape
    maxLabel = 228
    F1_scores = np.zeros(m,dtype = np.float)
   # y_tAll = np.zeros(maxLabel,dtype='i4')
   # y_pAll = np.zeros(maxLabel,dtype='i4')
    for i in range(1):
        image_id = dfPredict.loc[i,'image_id']
        if(type(dfPredict.loc[i,'label_id'])!=str):
          #  y_p = np.zeros(maxLabel,dtype ='i4')
            y_pred = []
        else:
            y_pred = strToArray((dfPredict.loc[i,'label_id']).split(' '))
        if(type(dfValidation.loc[i,'label_id'])!=str):
            y_true = []
        else:
            y_true = strToArray((dfValidation.loc[i,'label_id']).split(' '))
        y_t = np.zeros(maxLabel,dtype='i4')
        if(y_true != []):
            index = np.array(y_true)-1
            y_t[index] = 1
       # y_tAll =  y_tAll + y_t
        y_p = np.zeros(maxLabel,dtype ='i4')
        if(y_pred != []):
            index = np.array(y_pred) - 1
            y_p[index] = 1
        TP = np.sum(y_t * y_p)
       # print(TP)
        FP = np.sum(y_p) - TP
       # print(FP)
        FN = np.sum(np.maximum(y_t - y_p,0))
        precision = TP/(TP + FP)
        recall = TP/(TP + FN)
        F1_scores += precision * recall * 2 /(precision + recall  + 1e-10)
    f1 = np.mean(F1_scores)
    return f1

In [6]:
threshold = [0.507]
#(0.52, 0.469,)(0.54,0.543)(0.56,0.5789)(0.58,0.5789)(0.60,0.5789)(0.62,0.5833)(0.63,0.46)(0.60,0.464)(0.65,0.464)(0.61,0.34)
#(0.62,0.34)(0.65,0.28)
#new()
F1_Scores = []
for i in threshold:
    getPredictResult(i)
    f1 = getF1Score()
    F1_Scores.append(f1)
    print(f1)

  0%|                                                 | 0/1000 [00:00<?, ?it/s]

66 (score = 0.66786)
105 (score = 0.59210)
153 (score = 0.56643)
17 (score = 0.56124)
171 (score = 0.54620)
106 (score = 0.54563)
214 (score = 0.53746)
138 (score = 0.53308)
222 (score = 0.53095)


  0%|                                         | 1/1000 [00:02<44:53,  2.70s/it]

66 (score = 0.67368)
105 (score = 0.60426)
153 (score = 0.57285)
17 (score = 0.55896)
106 (score = 0.54978)
171 (score = 0.54516)
214 (score = 0.54355)
138 (score = 0.53635)
222 (score = 0.53595)


  0%|                                         | 2/1000 [00:03<35:19,  2.12s/it]

...

66 (score = 0.66994)
105 (score = 0.59781)
153 (score = 0.57030)
17 (score = 0.56031)
171 (score = 0.55464)
214 (score = 0.54715)
106 (score = 0.54331)
222 (score = 0.53751)
138 (score = 0.53384)


100%|██████████████████████████████████████▉| 999/1000 [14:26<00:00,  1.07it/s]

66 (score = 0.72897)
105 (score = 0.63460)
153 (score = 0.60673)
17 (score = 0.57420)
106 (score = 0.56303)
171 (score = 0.56258)
214 (score = 0.56161)
222 (score = 0.55063)
138 (score = 0.54795)


100%|██████████████████████████████████████| 1000/1000 [14:26<00:00,  1.07it/s]


0.193548387077


In [20]:
j = json.load(open("../data/origindata/validation.json"))
annotations = {}
annotations = j['annotations']
imageIds = []
labels = []
for annotation in annotations:
    imageIds.append(int(annotation['imageId']))
    l = annotation['labelId']
    label = ''.join([c + ' ' for c in l])
    a = label.rstrip()
    labels.append(a)
dataframe = pd.DataFrame({'image_id':imageIds,'label_id':labels})
dataframe.to_csv('validation.csv',sep = ',', index = False)

In [30]:
dfPredict = pd.read_csv("./predictResult.csv")
dfValidation = pd.read_csv("./validation.csv")
dfPredict = dfPredict.sort_values(by = 'image_id', ascending=True)
dfValidation = dfValidation.sort_values(by = 'image_id',ascending = True)
dfPredict.head()

,image_id,label_id
0,1,66 105 153 17 171 106 214 138 222
111,2,66 105 153 17 171 106 214 138 222 62 53
222,3,66 105 153 17 171
333,4,66 105 153 17 106 171
345,5,66 105 153 17 171 106 214


In [2]:
import tensorflow as tf
sess = tf.InteractiveSession()

In [24]:
y_t = tf.constant([[0,1],[0,1]])
y_p = tf.constant([[0,1],[0,1]])
y_t = tf.cast(y_t,tf.float32)
y_p = tf.cast(y_p, tf.float32)
TP = tf.reduce_sum(y_t * y_p,axis = 1,keep_dims = True)
FP = tf.reduce_sum(y_p, axis = 1,keep_dims = True) - TP
FN = tf.reduce_sum(tf.maximum(y_t - y_p,0), axis = 1,keep_dims = True)
precision = TP/(TP + FP + 1e-10 )
recall = TP/(TP + FN +1e-10)
F1_scores = precision * recall * 2 /(precision + recall  + 1e-10)
f1 = tf.reduce_mean(F1_scores)
print(sess.run(f1))
print(sess.run(TP))
print(sess.run(FP))
print(sess.run(FN))
print(sess.run(precision))
print(sess.run(recall))

0.0
[[ 0.]
 [ 0.]]
[[ 0.]
 [ 0.]]
[[ 1.]
 [ 1.]]
[[ 0.]
 [ 0.]]
[[ 0.]
 [ 0.]]


In [64]:
a = np.array([1,1,0,0])
b = np.array([0,1,1,0])
c =a * b
d = np.sum(c)
print(d)
d = np.mean(c)
d


1


0.25

In [33]:
1e-20

1e-20

In [6]:
print('Final f1_score accuracy = %.5f' % (0.34))

Final f1_score accuracy = 0.34000
